In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns

# from sklearn.preprocessing import train_test_split

ImportError: cannot import name 'train_test_split' from 'sklearn.preprocessing' (/home/sdatta/.local/lib/python3.8/site-packages/sklearn/preprocessing/__init__.py)